In [1]:
MONGO_URI="<Insert here MongoDB Atlas connection URI>"


from pyspark.sql import SparkSession
from pyspark.sql.functions import col

#Avvia la Spark Session mappando con delle confguration input e output da  verso MongoDB
spark = SparkSession \
    .builder \
    .appName("myApp") \
    .getOrCreate()

In [2]:
#Lettura del file csv
syn_dataset=spark.read.option("header", "false").option("inferschema","true").csv("/FileStore/tables/syn_dos_11000.csv")

#Conversione esplicita delle colonne in Double
syn_dataset_double = syn_dataset.select(*(col(c).cast("Double").alias(c) for c in syn_dataset.columns[:-1]),col(syn_dataset.columns[-1])).withColumnRenamed("_c115","Class")

In [3]:
#Scrittura su MongoDB
syn_dataset_double.write.format("mongo").option("spark.mongodb.output.uri", MONGO_URI).option("database", "test").option("collection", "test.syn-dos_double").save()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1602733774232489> in <module> () 
 1 #Scrittura su MongoDB 
 ----> 2 syn_dataset_double . write . format ( "mongo" ) . option ( "spark.mongodb.output.uri" , MONGO_URI ) . option ( "database" , "test" ) . option ( "collection" , "test.syn-dos_double" ) . save ( ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 734 self . format ( format ) 
 735 if path is None : 
 --> 736 self . _jwrite . save ( ) 
 737 else : 
 738 self . _jwrite . save ( path ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o643.save.
: java.lang.UnsupportedOperationException: MongoCollection already exists
	at com.mongodb.spark.sql.DefaultSource.createRelation(DefaultSource.scala:76)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:45)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:72)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:88)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:146)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:187)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:183)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:116)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:116)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:111)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:240)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:170)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:306)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:292)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.re